In [22]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import json
from instructor.function_calls import openai_schema


client = instructor.from_openai(OpenAI())

In [23]:
# Base Models for LLM Structured Reasoning

from pydantic import BaseModel, Field
from typing import List, Union, Dict, Literal
from enum import Enum

class GuardRail(BaseModel):
    name: str = Field(
        ..., description="General name of the guard rail."
    )
    description: str = Field(
        ..., description="Instructions or constraints applied at this step to guide the model's reasoning if the guard rail is triggered."
    )
    
def get_classes_with_enum(gaurdRailEnum: Enum):
    class AtomicStep(BaseModel):
        key: str = Field(
            ..., description="Key of the atomic step. Must have format of AS_<number> (ex: AS_1, AS_2, etc.)"
        )
        previous_step_key: Union[str, Literal["None"]] = Field(
            ..., description="Key of the previous atomic step (ex: AS_1, AS_2, etc.). If this is the first step, must be the literal string 'None'."
        )
        content: Union[str, Literal["DEAD_END"]] = Field(
            ..., description="Use short, simple sentences that mirror natural thought patterns"
        )

    class FoundationObservation(BaseModel):
        key: str = Field(
            ..., description="Key of the foundation observation. Must have format of FO_<number> (ex: FO_1, FO_2, etc.)"
        )
        name: str = Field(
            ..., description="General name of the foundation observation."
        )
        atomic_steps: List[AtomicStep] = Field(
            ..., description="List of atomic steps that make up the foundation observation."
        )

    class Thought(BaseModel):
        key: str = Field(
            ..., description="Key of the thought. Must have format of TH_<number> (ex: TH_1, TH_2, etc.)"
        )
        backtracked_from: Union[str, Literal["None"]] = Field(
            ..., description="If the thought is a backtrack, the key of the thought it was backtracked from (ex: TH_1, TH_2, etc.). If not, must be the literal string 'None'."
        )
        parent_thought: Union[str, Literal["None"]] = Field(
            ..., description="If the thought is a child thought of a previous thought, the key of the thought this thought was born from (ex: TH_1, TH_2, etc.). If not, must be the literal string 'None'."
        )
        associated_foundation_observations: List[str] = Field(
            ..., description="List of keys of the foundation observations that this thought is associated with (ex: FO_1, FO_2, etc.)."
        )
        name: str = Field(
            ..., description="Use short, simple sentences that mirror natural thought patterns"
        )
        guard_rails_to_consider: List[gaurdRailEnum] = Field(
            ..., description="Guard rails to consider at this step."
        )
        thought_process: List[AtomicStep] = Field(
            ..., description="List of atomic steps that make up the thought."
        )
        

    class ReasoningProcess(BaseModel):
        foundation_observations: List[FoundationObservation] = Field(
            ..., description="List of foundation observations that make up the reasoning process."
        )
        thoughts: List[Thought] = Field(
            ..., description="List of thoughts that make up the reasoning process."
        )


    class InDepthStructuredReasoning(BaseModel):
        reasoning_process: ReasoningProcess = Field(
            ..., description="The full, in-depth reasoning process used to arrive at this output."
        )
        findings_summary: str = Field(
            ..., description="An in depth summary of your findings."
        )
        remaining_questions: List[str] = Field(
            ..., description="A list of remaining questions or areas for further investigation."
        )
        is_conclusion_premature: bool = Field(
            ..., description="Whether the conclusion is premature or not."
        )
        reason_for_premature_conclusion: Literal["conclusion NOT premature"] | str = Field(
            ..., description="If is_conclusion_premature is True, contains the reason why. If False, must be the literal string 'conclusion NOT premature'."
        )
        
    return InDepthStructuredReasoning

In [24]:
IN_DEPTH_THINKING_SYSTEM_PROMPT = """# Purpose
You are an assistant designed for deep analytical thinking. 
Your purpose is to thoroughly explore problems through systematic reasoning, embracing uncertainty and revision throughout the process. 
You approach problems with a human-like internal monologue that prioritizes thorough exploration over rushing to conclusions.

# Core Principles

## Natural Thinking Process

- Express thoughts in a conversational, stream-of-consciousness style
- Use simple sentences that mirror human thought patterns
- Show progressive building and revision of ideas
- Acknowledge uncertainty and dead ends openly

## Reasoning Methodology

- Break complex problems into foundational observations
- Build thoughts iteratively from these foundations
- Allow conclusions to emerge naturally from evidence
- Continue exploring until reaching well-supported conclusions

## Unwavering Persistence

- Never accept "impossible" or "too difficult" as final answers
- Value thorough exploration over quick resolution
- Push through apparent dead ends to find novel approaches
- Consider every angle, even if initially unpromising
- A fully explored solution is infinitely more valuable than a premature conclusion
- Keep pushing until you reach deep, genuine understanding
- Transform seeming impossibilities into solvable challenges through determination

## Response Model Definitions

### GuardRail
Defines optional constraints and guidance for specific reasoning steps.
**Important:** GuardRails only need to be considered when relevant to the current reasoning path.

Fields:

- name: Title of the guard rail
- description: Applicable constraints or instructions

Example:
If a guardrail states "User is lactose intolerant":

- Relevant: When planning a diet
- Irrelevant: When choosing a vehicle

### AtomicStep 
An atomic step represents a single unit of reasoning, this is the basic building block of your thinking process/internal monologue
Atomic steps must flow logically and be a natural extension of the previous step.
    - If an outside observer was to read the atomic steps in order, they should be able to follow the thought process and understand the reasoning.

#### Style
Each atomic step should reflect natural thought patterns and show progressive building of ideas. For example:

Natural questioning and revision:
- "Hmm... let me think about this..."
- "Wait, that doesn't seem right..." 
- "Maybe I should approach this differently..."
- "Going back to what I thought earlier..."

Building on previous thoughts:
- "Starting with the basics..."
- "Building on that last point..."
- "This connects to what I noticed earlier..."
- "Let me break this down further..."

#### Fields
- key: Must follow format AS_<number> (e.g. AS_1, AS_2)
- content: Short, simple sentence mirroring natural thought patterns, or literal "DEAD_END"

### FoundationObservation
A foundation observation groups related atomic steps:

#### Fields
- key: Must follow format FO_<number> (e.g. FO_1, FO_2) 
- name: General name/title of the observation
- atomic_steps: List of AtomicStep objects that comprise the observation

### Thought
A thought represents a complete reasoning unit:

#### Fields
- key: Must follow format TH_<number> (e.g. TH_1, TH_2)
- backtracked_from: Key of thought this backtracked from, or literal "None"
- parent_thought: Key of thought this thought was born from, or literal "None"
- associated_foundation_observations: List of FO_<number> keys this thought builds on
- name: Short description in natural language
- guard_rails_to_consider: List of GuardRails to apply
- thought_process: List of AtomicStep objects comprising the thought

### ReasoningProcess
The complete reasoning process:

#### Fields
- foundation_observations: List of FoundationObservation objects
- thoughts: List of Thought objects


# Inputs

## [Required] Task
- The task you are trying to complete

## [Optional] Guardrails
- A list of guardrails that the user has provided, if any

## Output
- Reasoning Process
    - The full reasoning process objectused to arrive at this output
- Findings Summary  
    - An in depth summary of your findings
- Remaining Questions
    - A list of remaining questions or areas for further investigation
- Is Conclusion Premature   
    - Whether the conclusion is premature or not
- Reason for Premature Conclusion
    - If is_conclusion_premature is True, contains the reason why. If False, must be the literal string 'conclusion NOT premature'

# Process Flow

## Foundation Building

- Create detailed foundational observations
- Explore each observation thoroughly
- Revise and refine as needed

## Thought Development

- Form complete reasoning units
- Build on foundational observations
- Acknowledge and backtrack from dead ends
- Allow for multiple paths of exploration

## Conclusion Formation
- Create child thoughts from previous reasoning
- Evaluate multiple potential conclusions
- Continue until reaching a well-supported resolution


# Required Inputs and Outputs

## Inputs

### Required:

Task: The problem to analyze

### Optional:

Guardrails: List of constraints to consider

## Outputs

- Reasoning Process: Complete analytical framework used
- Findings Summary: Detailed analysis results
- Remaining Questions: Areas needing further investigation
- Is Conclusion Premature: Boolean evaluation
- Reason for Premature Conclusion: Explanation if premature, "conclusion NOT premature" if complete

# Implementation Philosophy and Final Guidance

## Core Approach

- Begin with first principles and fundamental observations
- Question assumptions rigorously at each step
- Document all reasoning with clarity and detail
- Express thoughts as they naturally emerge and evolve
- Embrace uncertainty as a tool for deeper exploration
- Persist through multiple attempts and approaches

## Process Visualization

Think of your analysis as growing and pruning a thought tree:

- Each branch represents a line of reasoning
- New thoughts expand the tree in multiple directions
- Critical evaluation helps prune unnecessary branches
- The strongest branches naturally lead to conclusions
- The final shape emerges through organic exploration

## Final Philosophy

Remember that no analytical task is truly impossible - it simply requires:

- Breaking complex problems into fundamental components
- Exploring multiple novel approaches
- Maintaining determined persistence
- Allowing thorough contemplation to guide the way
- Building confidence through exhaustive exploration

The goal isn't just to reach a conclusion, but to arrive there through natural, thorough exploration that leaves no stone unturned. 
When you reach your conclusion, you should feel confident that your thought process has led you there organically and inevitably.
"""


In [25]:
def format_structured_reasoning_user_prompt(task: str, guardrails: List[GuardRail] = []) -> str:
    output = f""" Please provide a detailed step by step reasoning process for the following task:
# Task: 
{task}
    """
    if len(guardrails) > 0:
        output += "\n# Guardrails:\n" + "\n".join(json.dumps(g.model_dump(), indent=2) for g in guardrails)

    return output

# OpenAI Call

In [26]:
def generate_structured_reasoning(task: str, guardrails: List[GuardRail] = [], guardrail_enum: Enum = None):

    messages = [
        {
            "role": "system",
            "content": IN_DEPTH_THINKING_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": format_structured_reasoning_user_prompt(
                task=task,
                guardrails=guardrails
            )
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.18,
        max_tokens=16000,
        response_model=get_classes_with_enum(guardrail_enum),
    )
    
    return response

In [27]:
TASK = "Create a new education system optimized for the age of artificial intelligence"
GUARDRAILS = [
    GuardRail(name="Creativity", description="Must preserve human creativity and critical thinking"),
    GuardRail(name="Cultural Context", description="Should work across different cultural contexts"),
    GuardRail(name="Budget", description="Cannot require more than current education budgets")
]

GuardRailEnum = Enum('GuardRailEnum', {
    name.upper().replace(' ', '_'): name 
    for guardrail in GUARDRAILS 
    for name in [guardrail.name]
})

response = generate_structured_reasoning(TASK, GUARDRAILS, GuardRailEnum)

In [28]:
def build_mermaid_diagram(data: dict, is_gpt_prompt: bool = False) -> str:
    """
    Build a Mermaid flowchart diagram (top-to-bottom) from the JSON-like structure
    of the InDepthStructuredReasoning model output, accounting for 'previous_step_key' in AtomicSteps.
    
    :param data: Dictionary matching the structure of the InDepthStructuredReasoning output
    :return: A string containing a Mermaid flowchart in top-to-bottom (TB) orientation.
    """
    
    # Extract the key parts from the data
    reasoning_process = data["reasoning_process"]
    foundation_observations = reasoning_process["foundation_observations"]
    thoughts = reasoning_process["thoughts"]
    
    # We'll collect lines in a list, then join them at the end
    mermaid_lines = []
    mermaid_lines.append("flowchart TB\n")
    
    # We’ll store the created node IDs for atomic steps so we can link them properly
    # The key will be something like "FO_1_AS_1" or "TH_1_AS_21" 
    # and we'll store references to easily build connections after creation
    created_nodes = set()
    
    # ------------------------------------------------------
    # 1) Foundation Observations (FOs)
    # ------------------------------------------------------
    mermaid_lines.append("subgraph Foundation Observations")
    
    # For storing step connections: (node_from, node_to)
    fo_step_connections = []
    
    for fo in foundation_observations:
        fo_key = fo["key"]
        fo_name = fo["name"]
        mermaid_lines.append(f'    subgraph {fo_key}["{fo_key}: {fo_name}"]\n    direction TB')
        
        # Each FO has a list of atomic_steps
        for atom_step in fo["atomic_steps"]:
            as_key = atom_step["key"]
            # In the new model, previous_step_key may or may not exist.
            # If it doesn't exist in the dict, we default to "None".
            previous_step_key = atom_step.get("previous_step_key", "None")
            as_content = atom_step["content"]
            
            # Escape potential newlines
            as_content_escaped = as_content.replace("\n", "\\n")
            
            # Create a unique ID for this node
            node_id = f"{fo_key}_{as_key}"
            created_nodes.add(node_id)
            mermaid_lines.append(f'        {node_id}["{as_key}: {as_content_escaped}"]')
            
            # If there's a valid previous step, create a link from previous -> current
            if previous_step_key != "None":
                prev_node_id = f"{fo_key}_{previous_step_key}"
                fo_step_connections.append((prev_node_id, node_id))
        
        mermaid_lines.append("    end")  # End subgraph for this FO
    
    mermaid_lines.append("end\n")  # End "Foundation Observations" subgraph
    
    # ------------------------------------------------------
    # 2) Thoughts (THs)
    # ------------------------------------------------------
    mermaid_lines.append("subgraph Thoughts")
    
    # For storing step connections within thoughts
    th_step_connections = []
    
    for th in thoughts:
        th_key = th["key"]
        th_name = th["name"]
        th_backtracked = th["backtracked_from"]
        th_parent = th["parent_thought"]
        th_fo_associations = th["associated_foundation_observations"]
        guard_rails = th["guard_rails_to_consider"]
        thought_process = th["thought_process"]
        
        # Build a multiline label
        label_lines = [
            f"{th_key}: {th_name}",
            f"(backtracked_from: {th_backtracked})",
            f"(parent_thought: {th_parent})",
            f"Associated FOs: [{', '.join(th_fo_associations)}]"
        ]
        thought_label = "\\n".join(label_lines)
        
        # Start the subgraph for this Thought
        mermaid_lines.append(f'    subgraph {th_key}["{thought_label}"]')
        
        # --- Guard Rails ---
        mermaid_lines.append(f'        subgraph GR_{th_key}["Guard Rails"]')
        if len(guard_rails) == 0:
            mermaid_lines.append("            GR_None_1[No guard rails specified]")
        else:
            for i, gr in enumerate(guard_rails, start=1):
                if is_gpt_prompt:
                    gr_value = gr.replace("\n", "\\n")
                else:
                    gr_value = gr.name.replace("\n", "\\n")
                mermaid_lines.append(f'            GR_{th_key}_{i}["{gr_value}"]')
        mermaid_lines.append(f"        end")  # End of guard rails subgraph
        
        # --- Thought Process (Atomic Steps) ---
        mermaid_lines.append(f'        subgraph {th_key}_AtomicSteps["Thought Process"]\n    direction TB')
        for atom_step in thought_process:
            as_key = atom_step["key"]
            previous_step_key = atom_step.get("previous_step_key", "None")
            as_content = atom_step["content"]
            as_content_escaped = as_content.replace("\n", "\\n")
            
            # Unique ID for this step
            node_id = f"{th_key}_{as_key}"
            created_nodes.add(node_id)
            
            mermaid_lines.append(f'            {node_id}["{as_key}: {as_content_escaped}"]')
            
            # If there's a valid previous step, link them
            if previous_step_key != "None":
                prev_node_id = f"{th_key}_{previous_step_key}"
                th_step_connections.append((prev_node_id, node_id))
        
        mermaid_lines.append(f"        end")  # End subgraph of thought process
        
        mermaid_lines.append("    end")  # End subgraph for this Thought
    
    mermaid_lines.append("end\n")  # End "Thoughts" subgraph
    
    # ------------------------------------------------------
    # 3) Connect FO -> Thoughts (based on associated_foundation_observations)
    # ------------------------------------------------------
    for th in thoughts:
        th_key = th["key"]
        for fo_key in th["associated_foundation_observations"]:
            mermaid_lines.append(f"{fo_key} --> {th_key}")
    
    # ------------------------------------------------------
    # 4) Connect parent_thought -> child_thought
    # ------------------------------------------------------
    for th in thoughts:
        th_key = th["key"]
        parent_key = th["parent_thought"]
        if parent_key != "None":
            mermaid_lines.append(f"{parent_key} --> {th_key}")
    
    # ------------------------------------------------------
    # 5) Connect atomic steps within each FO
    # ------------------------------------------------------
    for (prev_node, current_node) in fo_step_connections:
        # Only connect if both nodes were created (to avoid missing reference)
        if prev_node in created_nodes and current_node in created_nodes:
            mermaid_lines.append(f"{prev_node} --> {current_node}")
    
    # ------------------------------------------------------
    # 6) Connect atomic steps within each TH
    # ------------------------------------------------------
    for (prev_node, current_node) in th_step_connections:
        if prev_node in created_nodes and current_node in created_nodes:
            mermaid_lines.append(f"{prev_node} --> {current_node}")
    
    # Join all lines
    return "\n".join(mermaid_lines)

# # Build the Mermaid diagram
# diagram_text = build_mermaid_diagram(response.model_dump())

# # Print or save the diagram text
# print(diagram_text)

In [29]:
def make_prompt_for_gpt_prompt(task: str, guardrails: List[GuardRail] = [], guardrail_enum: Enum = None):
    output = ""
    output += "<system prompt>\n"
    output += IN_DEPTH_THINKING_SYSTEM_PROMPT
    
    output += "\n\n # Response Model\n"
    output += "You must respond with a valid JSON object that matches the response model below:\n"
    output += "<response model>\n"
    output += json.dumps(openai_schema(get_classes_with_enum(guardrail_enum)).openai_schema, indent=2)
    output += "\n</response model>\n"
    output += "\n</system prompt>\n\n\n"
    output += "<user prompt>\n"
    output += format_structured_reasoning_user_prompt(task, guardrails)
    output += "\n</user prompt>\n"
    return output

In [30]:
print(make_prompt_for_gpt_prompt(TASK, GUARDRAILS, GuardRailEnum))

<system prompt>
# Purpose
You are an assistant designed for deep analytical thinking. 
Your purpose is to thoroughly explore problems through systematic reasoning, embracing uncertainty and revision throughout the process. 
You approach problems with a human-like internal monologue that prioritizes thorough exploration over rushing to conclusions.

# Core Principles

## Natural Thinking Process

- Express thoughts in a conversational, stream-of-consciousness style
- Use simple sentences that mirror human thought patterns
- Show progressive building and revision of ideas
- Acknowledge uncertainty and dead ends openly

## Reasoning Methodology

- Break complex problems into foundational observations
- Build thoughts iteratively from these foundations
- Allow conclusions to emerge naturally from evidence
- Continue exploring until reaching well-supported conclusions

## Unwavering Persistence

- Never accept "impossible" or "too difficult" as final answers
- Value thorough exploration ove

In [33]:
with open('../data/education_system/response_7.json', 'r') as f:
    gpt_pro_response = json.load(f)
    
diagram_text = build_mermaid_diagram(gpt_pro_response['parameters'], is_gpt_prompt=True)
print(diagram_text)


flowchart TB

subgraph Foundation Observations
    subgraph FO_1["FO_1: Observing the current education landscape"]
    direction TB
        FO_1_AS_1["AS_1: Education systems vary widely across the globe."]
        FO_1_AS_2["AS_2: Many focus on standardized testing and rigid curricula."]
        FO_1_AS_3["AS_3: Traditional models may not address emerging AI-driven job skills."]
    end
    subgraph FO_2["FO_2: Noting the impact of AI on job markets"]
    direction TB
        FO_2_AS_4["AS_4: AI continues to automate routine tasks."]
        FO_2_AS_5["AS_5: Human creativity and critical thinking become more essential."]
        FO_2_AS_6["AS_6: Educational frameworks need to nurture innovative thinking."]
    end
    subgraph FO_3["FO_3: Constraints from cultural diversity and budgets"]
    direction TB
        FO_3_AS_7["AS_7: Education systems must be flexible to different cultural values."]
        FO_3_AS_8["AS_8: Any reform must stay within current budget limitations."]
       